In [ ]:
import numpy as np
import pandas as pd
import os
import glob

os.chdir("/root/autodl-tmp/SCALED-Particle")


# pred_files = sorted(glob.glob("exp/04_attn_sfc/rollout/npy/*.npy"))
# pred_files = sorted(glob.glob("exp/03_conv_sfc/rollout/npy/*.npy"))
# pred_files = sorted(glob.glob("exp/02_attn/rollout/npy/*.npy"))
pred_files = sorted(glob.glob("exp/01_conv/rollout/npy/*.npy"))
pred_arrays = [np.load(f) for f in pred_files]  # shape (8,256,64,64)

gt_files = sorted(glob.glob("data/evaluation_npy_step200-250/*.npy"))
gt_arrays = [np.load(f) for f in gt_files]  # shape (8,256,64,64)


def analyze_timesteps(gt_arrays, pred_arrays, output_csv="result/stat/stat.csv"):
    """
    Calculate statistics for each timestep of ground truth and prediction arrays
    Outputs a CSV file with rows containing gt, pred, and error metrics (diff, rel_err)
    """
    results = []

    # subdomain index
    x_start, x_end = 64//2 - 64 //4, 64//2 + 64 //4
    y_start, y_end = 64//2 - 64 //4, 64//2 + 64 //4
    z_start, z_end = 256//2 - 256 //4, 256//2 + 256 //4

    def compute_metrics(arr):
        """Calculate various metrics for a single timestep"""
        particle_vel = arr[0:3]  # (3, 256, 64, 64)
        fluid_vel = arr[3:6]     # (3, 256, 64, 64)
        mask = arr[7] > 0        # boolean mask

        # 1. particle velocity mean
        particle_vel_avg = []
        for i in range(3):
            vals = particle_vel[i][mask]
            particle_vel_avg.append(vals.mean() if vals.size > 0 else np.nan)

        # 2. fluid velocity mean
        fluid_vel_avg = [fluid_vel[i].mean() for i in range(3)]

        # 3. Reynolds stress
        u_fluct = [fluid_vel[i] - fluid_vel_avg[i] for i in range(3)]
        R_xx = np.mean(u_fluct[0] * u_fluct[0])
        R_yy = np.mean(u_fluct[1] * u_fluct[1])
        R_zz = np.mean(u_fluct[2] * u_fluct[2])
        R_xy = np.mean(u_fluct[0] * u_fluct[1])
        R_xz = np.mean(u_fluct[0] * u_fluct[2])
        R_yz = np.mean(u_fluct[1] * u_fluct[2])

        # 4. particle volume fraction (subdomain)
        sub_mask = mask[z_start:z_end, x_start:x_end, y_start:y_end]  # (256, 60, 60)
        vol_frac_per_layer = sub_mask.mean(axis=(1, 2))
        vol_frac_mean = vol_frac_per_layer.mean()
        vol_frac_std = vol_frac_per_layer.std()

        return {
            "particle_vel_avg_x": particle_vel_avg[0],
            "particle_vel_avg_y": particle_vel_avg[1],
            "particle_vel_avg_z": particle_vel_avg[2],
            "fluid_vel_avg_x": fluid_vel_avg[0],
            "fluid_vel_avg_y": fluid_vel_avg[1],
            "fluid_vel_avg_z": fluid_vel_avg[2],
            "fluid_Re_xx": R_xx,
            "fluid_Re_yy": R_yy,
            "fluid_Re_zz": R_zz,
            "fluid_Re_xy": R_xy,
            "fluid_Re_xz": R_xz,
            "fluid_Re_yz": R_yz,
            "particle_vol_frac_mean": vol_frac_mean,
            "particle_vol_frac_std": vol_frac_std
        }

    eps = 1e-8  # prevent division by zero

    # iterate over timesteps (assuming gt and pred have the same number of timesteps)
    for t_idx, (gt_arr, pred_arr) in enumerate(zip(gt_arrays, pred_arrays)):
        gt_metrics = compute_metrics(gt_arr)
        pred_metrics = compute_metrics(pred_arr)

        row = {"timestep": t_idx}
        # GT and Pred
        for k in gt_metrics:
            row[f"{k}_gt"] = gt_metrics[k]
            row[f"{k}_pred"] = pred_metrics[k]
            # error: difference & relative error
            row[f"{k}_diff"] = np.abs(pred_metrics[k] - gt_metrics[k])
            row[f"{k}_relerr"] = np.abs(pred_metrics[k] - gt_metrics[k]) / (np.abs(gt_metrics[k]) + eps)

        results.append(row)

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"Saved results to {output_csv}")
    return df

def summarize_errors(df, output_csv="comparison_summary.csv"):
    """
    Input a DataFrame of error metrics per timestep, output mean/max/std of each metric
    """
    error_cols = [c for c in df.columns if c.endswith("_diff") or c.endswith("_relerr")]
    summary = {}

    for col in error_cols:
        summary[col] = {
            "mean": df[col].mean(),
            "max": df[col].max(),
            "std": df[col].std()
        }

    summary_df = pd.DataFrame(summary).T  # row: metric, column: mean/max/std
    summary_df.to_csv(output_csv)
    print(f"Saved summary to {output_csv}")
    return summary_df


# df_detailed = analyze_timesteps(gt_arrays, pred_arrays)
# os.makedirs("result/stat", exist_ok=True)
# df_summary = summarize_errors(df_detailed, output_csv="result/stat/stat_summary.csv")


In [ ]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

methods = {
    "conv": "exp/01_conv/rollout/npy/*.npy",
    "attn": "exp/02_attn/rollout/npy/*.npy",
    "conv_sfc": "exp/03_conv_sfc/rollout/npy/*.npy",
    "attn_sfc": "exp/04_attn_sfc/rollout/npy/*.npy",
}

gt_files = sorted(glob.glob("data/evaluation_npy_step200-250/*.npy"))
gt_arrays = [np.load(f) for f in gt_files]

# for each method, generate summary csv
summary_dfs = {}
for method_name, pattern in methods.items():
    pred_files = sorted(glob.glob(pattern))
    pred_arrays = [np.load(f) for f in pred_files]
    
    df_detailed = analyze_timesteps(gt_arrays, pred_arrays, output_csv=f"result/stat/{method_name}_detailed.csv")
    df_summary = summarize_errors(df_detailed, output_csv=f"result/stat/{method_name}_summary.csv")
    summary_dfs[method_name] = df_summary

# -----------------------------
# plot multi-method bar chart
# -----------------------------
def plot_multi_method(summary_dfs, metrics=None, metric_type="diff", use_std=False, logscale=False):
    """
    summary_dfs: dict, {method_name: summary_df}
    metrics: list, specify metrics to plot (excluding _diff/_relerr)
    metric_type: "diff" or "relerr"
    use_std: whether to plot std
    """
    method_names = list(summary_dfs.keys())
    n_methods = len(method_names)
    
    # default to plot all metrics
    if metrics is None:
        metrics = [c.replace(f"_{metric_type}","") for c in summary_dfs[method_names[0]].index if metric_type in c]
    
    x = np.arange(len(metrics))
    width = 0.18  # width of each bar
    fig, ax = plt.subplots(figsize=(16,6))
    
    colors = ["skyblue", "salmon", "limegreen", "orange"][:n_methods]
    
    for i, method in enumerate(method_names):
        df = summary_dfs[method]
        col_names = [f"{m}_{metric_type}" for m in metrics]
        means = df.loc[col_names, "mean"].values
        if use_std:
            errs = df.loc[col_names, "std"].values
        else:
            errs = None
        ax.bar(x + i*width, means, width, yerr=errs, capsize=3, label=method, color=colors[i])
    
    if logscale:
        ax.set_yscale("log")
    ax.set_xticks(x + width*(n_methods-1)/2)
    ax.set_xticklabels(metrics, rotation=45, ha="right")
    ax.set_ylabel("Absolute Difference (pred - gt)" if metric_type=="diff" else "Relative Error")
    ax.set_title(f"Comparison of {metric_type} Across Methods")
    ax.legend()
    plt.tight_layout()
    plt.savefig(f"result/stat/multi_method_{metric_type}.png")
    plt.show()

# plot diff
plot_multi_method(summary_dfs, metric_type="diff", use_std=False)

# plot relative error
# plot_multi_method(summary_dfs, metric_type="relerr", use_std=False, logscale=True)
